In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import BeamDynamics as bd
import copy

In [ ]:
from importlib import reload
reload(bd)

In [ ]:
%connect_info

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
plt.rcParams['figure.figsize'] = [9.6, 6.4]
defaultColorCycle = plt.rcParams["axes.prop_cycle"].by_key()['color']

# FODO

## Analytical relations from thin lens approximation

### General relations

In [ ]:
def fodo_thin_lcell(betaPlus, psi):
    Lcell = betaPlus * np.sin(psi) / (1 + np.sin(psi/2.))
    return Lcell

def fodo_thin_lcell_2(f, psi):
    Lcell = 4 * f * np.sin(psi/2.)
    return Lcell

def fodo_thin_betaminus(Lcell, psi):
    betaMinus = Lcell * (1 - np.sin(psi/2.)) / np.sin(psi)
    return betaMinus

def fodo_thin_f(Lcell, psi):
    f = Lcell / (4 * np.sin(psi/2.))
    return f

def fodo_thin_lquad_1(f, kQuad):
    Lquad = 1 / (kQuad * f)
    return Lquad

def fodo_thin_kquad(f, Lquad):
    kQuad = 1. / (Lquad * f)
    return kQuad

def fodo_thin_ldrift_1(Lcell, Lquad):
    Ldrift = (Lcell - 2.*Lquad) / 2.
    return Ldrift

def fodo_thin_lquad_2(Lcell, Ldrift):
    Lquad = (Lcell - 2.*Ldrift) / 2.
    return Lquad

### Assumption: BetaMax determined by emittance and aperture

In [ ]:
def fodo_thin_betaplus(Ra, Fa, gammaRel, emitn):
    betaRel = bd.gamma_to_beta(gammaRel)
    betaPlus = (Ra/Fa)**2. * betaRel * gammaRel / emitn
    return betaPlus

### Assumption: Using the max. available quad gradient

In [ ]:
def fodo_thin_ldrift_2(Ra, Fa, gammaRel, emitn, psi, Gquad):
    betaPlus = fodo_thin_betaplus(Ra, Fa, gammaRel, emitn)
    term1 = betaPlus * np.sin(psi) / (2.*(1.+np.sin(psi/2.)))
    kQuad = bd.quad_strength(Gquad, bd.gamma_to_p(gammaRel, -11))
    term2 = 4. * np.sin(psi/2.) * (1.+np.sin(psi/2.)) / (betaPlus * np.sin(psi) * kQuad)
    Ldrift = term1 - term2
    return Ldrift

### Assumption: Limit aperture to length ratio of the quad

In [ ]:
def fodo_thin_lquad_3(f, quadGradMax, p, Rpole, RpoleToLquadMaxRatio):
    kQuadMax = bd.quad_strength(quadGradMax, p)
    LquadMin = 1 / (kQuadMax * f)
    takeLquadMinInds = Rpole / LquadMin < RpoleToLquadMaxRatio
    Lquad = np.zeros(takeLquadMinInds.shape)
    Lquad[takeLquadMinInds] = LquadMin[takeLquadMinInds]
    Lquad[~takeLquadMinInds] = Rpole / RpoleToLquadMaxRatio
    return Lquad

## Results from Elegant optimizations (thick lenses)

In [ ]:
data = {
    'Ra': [
             20.,      20.,      20.,      20.,      20.,      20.,      20.,
             25.,      30.,      35.,      40.,      45.,
             30.,
    ],   # [mm]
    'Fa': [
              3.,       3.,       3.,       3.,       3.,       3.,       3.,
              4.,       4.,       4.,       4.,       4.,
              4.,
    ],
    'psi': [
            22.5,      45.,      60.,      70.,   76.345,      90.,     100.,
          76.345,   76.345,   76.345,   76.345,   76.345,
          76.345,
    ],   # [deg]
    'LquadHalf' : [
        0.107503, 0.112642, 0.129234, 0.144463, 0.156520, 0.193600, 0.256245,
        0.203432, 0.135236, 0.079521, 0.059732, 0.046750,
        0.5,
    ],   # [m]
    'Ldrift': [
        0.070826, 0.235100, 0.266819, 0.259910, 0.243498, 0.169696, 0.034461,
        0.094104, 0.408817, 0.759710, 1.071240, 1.407448,
        0.757640,
    ],   # [m]
    'p': [
            200.,     200.,     200.,     200.,     200.,     200.,     200.,
            200.,     200.,     200.,     200.,     200.,
            500.,
    ],
}
fodoDf = pd.DataFrame(data=data)
fodoDf['Lcell'] = 2.*2.*fodoDf['LquadHalf'] + 2.*fodoDf['Ldrift']

## Cell length vs phase advance

### Fix parameters

In [ ]:
Ra = 20.   # [mm]
Fa = 3.
p = 200.   # [MeV/c]
gammaRel = bd.p_to_gamma(p, -11)
emitn = 10e3   # [pi mm mrad]
betaPlus = fodo_thin_betaplus(Ra, Fa, gammaRel, emitn)
Gquad = 0.6 / 0.1   # [T/m]
kQuad = bd.quad_strength(Gquad, p)
print('gammaRel = {:.1f}.'.format(gammaRel))
print('betaPlus = {:.3f} m.'.format(betaPlus))
print('kQuad = {:.3f} 1/m2.'.format(kQuad))

### Select Elegant results

In [ ]:
fodoSel = fodoDf[(fodoDf['Fa']==Fa) & (fodoDf['p']==p)]

### Variable parameter: Phase advance psi

In [ ]:
psi = np.linspace(fodoSel['psi'].min(), fodoSel['psi'].max())

### Compute curves from thin lens approx.

In [ ]:
LcellThin = fodo_thin_lcell(betaPlus, psi/180.*np.pi)
LquadThin = fodo_thin_lquad_1(fodo_thin_f(LcellThin, psi/180.*np.pi), kQuad)
LdriftThin = fodo_thin_ldrift_1(LcellThin, LquadThin)

### Compare thick lenses (Elegant) vs. thin lens approx.

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8,12))
ax[0].plot(fodoSel['psi'], fodoSel['Lcell'], 'o-')
ax[0].plot(psi, LcellThin, '--')
ax[0].grid()
ax[0].set_ylim((0, 1.2))
ax[0].set_xlabel('Phase advance [deg]')
ax[0].set_ylabel('FODO cell length [m]')
ax[0].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=8)
ax[1].plot(fodoSel['psi'], fodoSel['Ldrift'], 'o-')
ax[1].plot(psi, LdriftThin, '--')
ax[1].grid()
ax[1].set_ylim((0, 0.3))
ax[1].set_xlabel('Phase advance [deg]')
ax[1].set_ylabel('Drift length [m]')
ax[1].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=8)
ax[2].plot(fodoSel['psi'], fodoSel['LquadHalf']*2., 'o-')
ax[2].plot(psi, LquadThin, '--')
ax[2].grid()
ax[2].set_ylim((0, 0.6))
ax[2].set_xlabel('Phase advance [deg]')
ax[2].set_ylabel('Quad length [m]')
_ = ax[2].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=8)

## Cell length vs. aperture

### Fix parameters

In [ ]:
Fa = 4.
p = 200.   # [MeV/c]
gammaRel = bd.p_to_gamma(p, -11)
emitn = 10e3   # [pi mm mrad]
Gquad = 0.6 / 0.1   # [T/m]
kQuad = bd.quad_strength(Gquad, p)
psi = 76.345 / 180. * np.pi   # [rad]
print('gammaRel = {:.1f}.'.format(gammaRel))
print('kQuad = {:.3f} 1/m2.'.format(kQuad))

### Select Elegant results

In [ ]:
fodoSel = fodoDf[(fodoDf['Fa']==Fa) & (fodoDf['p']==p)]

### Variable parameter

In [ ]:
Ra = np.linspace(20., fodoSel['Ra'].max())

### Compute curves from thin lens approx.

In [ ]:
LdriftThin = fodo_thin_ldrift_2(Ra, Fa, gammaRel, emitn, psi, Gquad)
LcellThin = fodo_thin_lcell(fodo_thin_betaplus(Ra, Fa, gammaRel, emitn), psi)
LquadThin = fodo_thin_lquad_2(LcellThin, LdriftThin)

### Compare thick lenses (Elegant) vs. thin lens approx.

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8,12))
ax[0].plot(fodoSel['Ra'], fodoSel['Lcell'], 'o-')
ax[0].plot(Ra, LcellThin, '--')
ax[0].grid()
ax[0].set_ylim((0, 3.5))
ax[0].set_xlabel('Aperture (iris radius) [mm]')
ax[0].set_ylabel('FODO cell length [m]')
ax[0].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=9)
ax[1].plot(fodoSel['Ra'], fodoSel['Ldrift'], 'o-')
ax[1].plot(Ra, LdriftThin, '--')
ax[1].grid()
ax[1].set_ylim((-0.2, 1.6))
ax[1].set_xlabel('Aperture (iris radius) [mm]')
ax[1].set_ylabel('Drift length [m]')
ax[1].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=9)
ax[2].plot(fodoSel['Ra'], fodoSel['LquadHalf']*2., 'o-')
ax[2].plot(Ra, LquadThin, '--')
ax[2].grid()
ax[2].set_ylim((0, 0.5))
ax[2].set_xlabel('Aperture (iris radius) [mm]')
ax[2].set_ylabel('Quad length [m]')
_ = ax[2].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=9)

## Maximize drift length

### Fix parameters

In [ ]:
Fa = 4.
emitn = 10e3   # [pi mm mrad]
Gquad = 1.0 / 0.1   # [T/m]

### Discrete parameters

In [ ]:
pArr = np.array((200., 350., 500.))   # [MeV/c]
gammaRelArr = bd.p_to_gamma(pArr, -11)
kQuadArr = bd.quad_strength(Gquad, pArr)

### Variable parameters

In [ ]:
Ra = np.linspace(20., 40.)
psi = np.linspace(10., 90.) / 180. * np.pi   # [rad]
Ra, psi = np.meshgrid(Ra, psi)
psiDeg = psi / np.pi * 180

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), subplot_kw={"projection": "3d"})
for p, gammaRel, kQuad in zip(pArr, gammaRelArr, kQuadArr):
    # Compute curves from thin lens approx.
    LdriftThin = fodo_thin_ldrift_2(Ra, Fa, gammaRel, emitn, psi, Gquad)
    LcellThin = fodo_thin_lcell(fodo_thin_betaplus(Ra, Fa, gammaRel, emitn), psi)
    LquadThin = fodo_thin_lquad_2(LcellThin, LdriftThin)
    # Plot
    surf = ax.plot_surface(
        Ra, psiDeg, LdriftThin,
        cmap=cm.coolwarm, vmin=0, vmax=2.7,
        linewidth=0, rstride=5, cstride=5, antialiased=False
    )
fig.colorbar(surf, shrink=0.3, aspect=15)
ax.set_xlim(np.min(Ra), np.max(Ra))
ax.set_ylim(np.min(psiDeg), np.max(psiDeg))
ax.set_xlabel('Ra [mm]')
ax.set_ylabel('Phase advance [deg]')
ax.set_zlabel('Ldrift [m]')
ax.view_init(elev=35., azim=20.)
ax.text(32.5, 80., 0, 'p = 200 MeV/c', (1,0,0.05), zorder=4)
ax.text(32.5, 80., 0.9, 'p = 350 MeV/c', (1,0,0.09), zorder=5)
_ = ax.text(37.5, 85., 2.5, 'p = 500 MeV/c', (1,0,0.11), zorder=6)

## Select working points

### Fix parameters

#### Section 2

In [ ]:
# Ra = 30.   # [mm]
# Fa = 4.
# p = 500.   # [MeV/c]
# gammaRel = bd.p_to_gamma(p, -11)
# emitn = 10e3   # [pi mm mrad]
# betaPlus = fodo_thin_betaplus(Ra, Fa, gammaRel, emitn)
# Rpole = 0.1   # [m]
# Gquad = 1. / Rpole   # [T/m]
# RpoleToLquadMaxRatio = 0.1
# print('gammaRel = {:.1f}.'.format(gammaRel))
# print('betaPlus = {:.3f} m.'.format(betaPlus))

#### Section 3

In [ ]:
Ra = 20.   # [mm], beam pipe aperture
Fa = 4.
p = 1000.   # [MeV/c]
gammaRel = bd.p_to_gamma(p, -11)
emitn = 10e3   # [pi mm mrad]
betaPlus = fodo_thin_betaplus(Ra, Fa, gammaRel, emitn)
Rpole = 0.025   # [m]
Gquad = 1. / Rpole   # [T/m]
RpoleToLquadMaxRatio = 0.1
print('gammaRel = {:.1f}.'.format(gammaRel))
print('betaPlus = {:.3f} m.'.format(betaPlus))

### Select Elegant results

In [ ]:
fodoSel = fodoDf.iloc[-1,:]

### Variable parameter: Phase advance psi

In [ ]:
psiDeg = np.linspace(10., 90.)   # [deg]
psi = psiDeg / 180. * np.pi   # [rad]

### Compute curves from thin lens approx.

In [ ]:
LcellThin = fodo_thin_lcell(betaPlus, psi)
LquadThin = fodo_thin_lquad_1(fodo_thin_f(LcellThin, psi), kQuad)
LquadRealMagnet = fodo_thin_lquad_3(fodo_thin_f(LcellThin, psi), Gquad, p, Rpole, RpoleToLquadMaxRatio)
LdriftThin = fodo_thin_ldrift_1(LcellThin, LquadThin)
LdriftRealMagnet = fodo_thin_ldrift_1(LcellThin, LquadRealMagnet)

### Compare thick lenses (Elegant) vs. thin lens approx.

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(9.6,12))
ax[0].plot(fodoSel['psi'], fodoSel['Lcell'], 'o-')
ax[0].plot(psiDeg, LcellThin, '--')
ax[0].grid()
ax[0].set_ylim((0, 6.))
ax[0].set_xlabel('Phase advance [deg]')
ax[0].set_ylabel('FODO cell length [m]')
ax[0].legend(('Elegant optim.', 'Analytical (thin lens)'), loc=8)
ax[1].plot(fodoSel['psi'], fodoSel['Ldrift'], 'o-')
ax[1].plot(psiDeg, LdriftThin, '--')
ax[1].plot(psiDeg, LdriftRealMagnet, '--')
ax[1].grid()
ax[1].set_ylim((-0.5, 3.0))
ax[1].set_xlabel('Phase advance [deg]')
ax[1].set_ylabel('Quad spacing [m]')
ax[1].legend(('Elegant optim.', 'Analytical (thin lens)', 'Analytical (limit Rpole/Lquad ratio)'), loc=8)
ax[2].plot(fodoSel['psi'], fodoSel['LquadHalf']*2., 'o-')
ax[2].plot(psiDeg, LquadThin, '--')
ax[2].plot(psiDeg, LquadRealMagnet, '--')
ax[2].grid()
ax[2].set_ylim((0, 1.1))
ax[2].set_xlabel('Phase advance [deg]')
ax[2].set_ylabel('Min. quad length [m]')
_ = ax[2].legend(('Elegant optim.', 'Analytical (thin lens)', 'Analytical (limit Rpole/Lquad ratio)'), loc=8)

## Beta Function and Beam Size vs. Energy

### Relations Computing the Beam Size

In [ ]:
def sigma_from_twiss(betaTwiss, emitn, gammaRel):
    sigma = np.sqrt(betaTwiss * emitn / (gammaRel*bd.gamma_to_beta(gammaRel))) / 1e3   # [m]
    return sigma

### Fix Parameters

In [ ]:
# Ra = 30.   # [mm], beam pipe aperture
# Fa = 4.
# pStart = 500.   # [MeV/c]
# emitn = 10e3   # [pi mm mrad]
# psiDeg = 76.345   # [deg]
# gammaRelStart = bd.p_to_gamma(pStart, -11)
# betaPlusStart = fodo_thin_betaplus(Ra, Fa, gammaRelStart, emitn)
# print('gammaRelStart = {:.1f}.'.format(gammaRelStart))
# print('betaPlusStart = {:.3f} m.'.format(betaPlusStart))

In [ ]:
Ra = 20.   # [mm], beam pipe aperture
Fa = 4.
pStart = 500.   # [MeV/c]
emitn = 10e3   # [pi mm mrad]
psiDeg = 76.345   # [deg]
gammaRelStart = bd.p_to_gamma(pStart, -11)
betaPlusStart = fodo_thin_betaplus(Ra, Fa, gammaRelStart, emitn)
print('gammaRelStart = {:.1f}.'.format(gammaRelStart))
print('betaPlusStart = {:.3f} m.'.format(betaPlusStart))

### Variable Parameters

In [ ]:
p = np.linspace(pStart, 1500.)
gammaRel = bd.p_to_gamma(p, -11)

### Compute Curves from Thin Lens Approximation

$$ \beta^+_{x,lim} = \left( \frac{R_a}{F_a} \right)^2 \frac{\gamma_r \beta_r}{\epsilon_{norm,x}} $$

$$ L_{cell} = \beta^\pm_x \frac{\sin \psi}{1 \pm \sin \frac{\psi}{2}} = 4 * f * \sin \frac{\psi}{2} $$

$$ f = \frac{1}{k_{quad} \cdot L_{quad}} $$

$$ \sigma_x = \sqrt{\beta_x \epsilon_{geom,x}} = \sqrt{\frac{\beta_x \epsilon_{norm,x}}{\gamma_r \beta_r}} $$

In [ ]:
betaPlusLim = fodo_thin_betaplus(Ra, Fa, gammaRel, emitn)
LcellLim = fodo_thin_lcell(betaPlusLim, psiDeg/180.*np.pi)
betaMinusLim = fodo_thin_betaminus(LcellLim, psiDeg/180.*np.pi)

In [ ]:
pStart2 = 500.
pEnd2 = 1000.
Rpole2 = 0.1   # [m]
GquadMax2 = 1. / Rpole2   # [T/m]
pSection2 = np.linspace(pStart2, pEnd2)
gammaRel2 = bd.p_to_gamma(pSection2, -11)
betaPlusActual2 = np.full(pSection2.shape, np.interp(pStart2, p, betaPlusLim))
betaMinusActual2 = np.full(pSection2.shape, np.interp(pStart2, p, betaMinusLim))
sigmaPlusActual2 = sigma_from_twiss(betaPlusActual2, emitn, gammaRel2)
sigmaMinusActual2 = sigma_from_twiss(betaMinusActual2, emitn, gammaRel2)
LcellActual2 = fodo_thin_lcell(betaPlusActual2, psiDeg/180.*np.pi)
fActual2 = fodo_thin_f(LcellActual2, psiDeg/180.*np.pi)
LquadActual2 = fodo_thin_lquad_3(fActual2, GquadMax2, pSection2, Rpole2, RpoleToLquadMaxRatio)
kQuadActual2 = fodo_thin_kquad(fActual2, LquadActual2)
GquadActual2 = bd.quad_gradient(kQuadActual2, pSection2)
LdriftActual2 = fodo_thin_ldrift_1(LcellActual2, LquadActual2)

In [ ]:
pStart3 = 1000.
pEnd3 = 1500.
pSection3 = np.linspace(pStart3, pEnd3)
gammaRel3 = bd.p_to_gamma(pSection3, -11)
# Rpole3 = 0.035   # [m]
Rpole3 = 0.025   # [m]
GquadMax3 = 1. / Rpole3   # [T/m]
betaPlusActual3 = np.full(pSection3.shape, np.interp(pStart3, p, betaPlusLim))
betaMinusActual3 = np.full(pSection3.shape, np.interp(pStart3, p, betaMinusLim))
sigmaPlusActual3 = sigma_from_twiss(betaPlusActual3, emitn, gammaRel3)
sigmaMinusActual3 = sigma_from_twiss(betaMinusActual3, emitn, gammaRel3)
LcellActual3 = fodo_thin_lcell(betaPlusActual3, psiDeg/180.*np.pi)
fActual3 = fodo_thin_f(LcellActual3, psiDeg/180.*np.pi)
LquadActual3 = fodo_thin_lquad_3(fActual3, GquadMax3, pSection3, Rpole3, RpoleToLquadMaxRatio)
kQuadActual3 = fodo_thin_kquad(fActual3, LquadActual3)
GquadActual3 = bd.quad_gradient(kQuadActual3, pSection3)
LdriftActual3 = fodo_thin_ldrift_1(LcellActual3, LquadActual3)

### Plot

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(9.6, 16))
bplL = ax[0].plot(p, betaPlusLim, 'k-', label=r'$\beta^+_{x,lim}$')
bmlL = ax[0].plot(p, betaMinusLim, 'k--', label=r'$\beta^-_{x,lim}$')
ax[0].fill_between(
    pSection2, betaPlusActual2, betaMinusActual2,
    color=defaultColorCycle[3], alpha=0.5, label=r'$\beta_{x,Section2}$'
)
ax[0].plot(pSection2, betaPlusActual2, '-', color=defaultColorCycle[3], label=r'$\beta^+_{x,Section2}$')
ax[0].plot(pSection2, betaMinusActual2, '--', color=defaultColorCycle[3], label=r'$\beta^-_{x,Section2}$')
ax[0].fill_between(
    pSection3, betaPlusActual3, betaMinusActual3,
    color=defaultColorCycle[0], alpha=0.5, label=r'$\beta_{x,Section3}$'
)
ax[0].plot(pSection3, betaPlusActual3, '-', color=defaultColorCycle[0], label=r'$\beta^+_{x,Section3}$')
ax[0].plot(pSection3, betaMinusActual3, '--', color=defaultColorCycle[0], label=r'$\beta^-_{x,Section3}$')
ax[0].set_xlabel('Beam momentum [MeV/c]')
ax[0].set_ylabel('Beta function')
ax[0].legend()
ax[0].grid()
ax[1].fill_between(
    pSection2, sigmaPlusActual2*1e3, sigmaMinusActual2*1e3,
    color=defaultColorCycle[3], alpha=0.5, label=r'$\sigma_{x,Section2}$'
)
ax[1].plot(pSection2, sigmaPlusActual2*1e3, '-', color=defaultColorCycle[3], label=r'$\sigma^+_{x,Section2}$')
ax[1].plot(pSection2, sigmaMinusActual2*1e3, '--', color=defaultColorCycle[3], label=r'$\sigma^-_{x,Section2}$')
ax[1].fill_between(
    pSection3, sigmaPlusActual3*1e3, sigmaMinusActual3*1e3,
    color=defaultColorCycle[0], alpha=0.5, label=r'$\sigma_{x,Section3}$'
)
ax[1].plot(pSection3, sigmaPlusActual3*1e3, '-', color=defaultColorCycle[0], label=r'$\sigma^+_{x,Section3}$')
ax[1].plot(pSection3, sigmaMinusActual3*1e3, '--', color=defaultColorCycle[0], label=r'$\sigma^-_{x,Section2}$')
ax[1].set_xlim(ax[0].get_xlim())
ax[1].set_ylim((0, 8.))
ax[1].set_xlabel('Beam momentum [MeV/c]')
ax[1].set_ylabel('Sigma beam [mm]')
ax[1].legend()
ax[1].grid()
ax[2].plot(pSection2, LdriftActual2, '-', color=defaultColorCycle[3], label='Quad spacing, Section 2')
ax[2].plot(pSection3, LdriftActual3, '-', color=defaultColorCycle[0], label='Quad spacing, Section 3')
ax[2].plot(pSection2, LquadActual2, '--', color=defaultColorCycle[3], label='Quad length, Section 2')
ax[2].plot(pSection3, LquadActual3, '--', color=defaultColorCycle[0], label='Quad length, Section 3')
ax[2].set_xlim(ax[0].get_xlim())
ax[2].set_ylim((-0.5, 4.))
ax[2].set_xlabel('Beam momentum [MeV/c]')
ax[2].set_ylabel('Quadrupole spacing / length [m]')
ax[2].legend()
ax[2].grid()
ax[3].plot(pSection2, kQuadActual2, '-', color=defaultColorCycle[3], label='Quad strength, Section 2')
ax[3].plot(pSection3, kQuadActual3, '-', color=defaultColorCycle[0], label='Quad strength, Section 3')
ax[3].set_xlim(ax[0].get_xlim())
ax[3].set_ylim((0, 4.0))
ax[3].set_xlabel('Beam momentum [MeV/c]')
ax[3].set_ylabel('Quad strength [1/m2] (solid)')
ax[3].legend()
ax[3].grid()
ax3b = ax[3].twinx()
ax3b.plot(pSection2, GquadActual2, '--', color=defaultColorCycle[3], label='Quad gradient, Section 2')
ax3b.plot(pSection3, GquadActual3, '--', color=defaultColorCycle[0], label='Quad gradient, Section 3')
ax3b.set_ylim((0, 20.))
ax3b.set_ylabel('Quad gradient [T/m] (dashed)')

In [ ]:
15.*0.025

In [ ]:
kQuadActual3

## Cost Model

In [ ]:
Eaccel = 15.   # [MV/m]
L2 = (pEnd2 - pStart2) / Eaccel
Nquads2 = np.round(L2 / LcellActual2 * 2.)
print('L2 = {:.3f} m.'.format(L2))
print('Nquads2 = {:.0f}.'.format(Nquads2[0]))
L3 = (pEnd3 - pStart3) / Eaccel
Nquads3 = np.round(L3 / LcellActual3 * 2.)
print('L3 = {:.3f} m.'.format(L3))
print('Nquads3 = {:.0f}.'.format(Nquads3[0]))

## Input Parameters

In [ ]:
emitnSelected = 10e3   # [pi mm mrad]
pSelected = 500.   # [MeV/c]
RaSelected = 30.   # [mm]
FaSelected = 4.
psiDegSelected = 76.345   # [deg]
LquadSelected = 1.

In [ ]:
kQuadElegantOptim = 0.894860   # [1/m2]

### Elegant

In [ ]:
gammaRelSelected = bd.p_to_gamma(pSelected, -11)
betaRelSelected = bd.p_to_beta(pSelected, -11)
gammaRelSelected * betaRelSelected

In [ ]:
betaPlusSelected = fodo_thin_betaplus(RaSelected, FaSelected, gammaRelSelected, emitnSelected)
betaPlusSelected

In [ ]:
psiSelected = psiDegSelected / 180. * np.pi
LcellSelected = fodo_thin_lcell(betaPlusSelected, psiSelected)
LcellSelected

In [ ]:
betaMinusSelected = fodo_thin_betaminus(LcellSelected, psiSelected)
betaMinusSelected

In [ ]:
LdriftSelected = fodo_thin_ldrift_1(LcellSelected, LquadSelected)
LdriftSelected

In [ ]:
kQuadSelected = fodo_thin_kquad(fodo_thin_f(LcellSelected, psiSelected), LquadSelected)
kQuadSelected

## Placet

In [ ]:
EkinSelected = bd.p_to_Ekin(pSelected, -11)
EkinSelected

In [ ]:
betaRelSelected

In [ ]:
fQuadSelected = fodo_thin_f(LcellSelected, psiSelected)
fQuadSelected

In [ ]:
fQuadElegantOptim = 1. / (kQuadElegantOptim * LquadSelected)
fQuadElegantOptim

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some error.
</div>

<div class="alert alert-block alert-success">
Something good.
</div>